# Práctica 1: Buscador equivalencias fonéticas en un corpus

Elaborado por: Alejandro Axel Rodríguez Sánchez  
Correo: [ahexo@ciencias.unam.mx](mailto:ahexo@ciencias.unam.mx)  
Github: [@Ahexo](https://github.com/Ahexo/)  
Número de Cuenta: 315247697  
Institución: Facultad de Ciencias UNAM  
Asignatura: Lingüística computacional  
Grupo: 7014  
Semestre: 2024-1  

---

## Especificación de la práctica

1. Agregar un nuevo modo de búsqueda donde se extienda el comportamiento básico del buscador para ahora buscar por frases.

```
[es_MX]>>> Hola que hace
 /ola/ /ke/ /ase/
```

2. Optimizar el código para agregar los datasets en un "cache" a demanda y no descargar todo el corpus de un trancazo. Esto quiere decir que al inicio de la ejecución no habrá ningun dataset descargado. Mientras la usuaria vaya seleccionado idiomas los irá agregando a un cache local (puede ser persistente o en memoria):

```
lang>> es_MX
Corpus no encontrado. Descargando ...
[es_MX]>>
...
lang>> en_US
Corpus no encontrado. Descargando ...
[en_US]>>
...
lang>> es_MX
[es_MX]>>
...
```

3. (Extra) Mejorar la solución al escenario cuando no se encuentran las palabras en el dataset mostrando palabras similares. Ejemplo:

```
[es_MX]>> pero
No se encontro <<pero>> en el dataset. Palabras aproximadas:
perro /pero/
perno /peɾno/
[es_MX]>>
```

---


## Obteniendo el corpus

In [1]:
lang_codes = {
  "ar": "Arabic (Modern Standard)",
  "de": "German",
  "en_UK": "English (Received Pronunciation)",
  "en_US": "English (General American)",
  "eo": "Esperanto",
  "es_ES": "Spanish (Spain)",
  "es_MX": "Spanish (Mexico)",
  "fa": "Persian",
  "fi": "Finnish",
  "fr_FR": "French (France)",
  "fr_QC": "French (Québec)",
  "is": "Icelandic",
  "ja": "Japanese",
  "jam": "Jamaican Creole",
  "km": "Khmer",
  "ko": "Korean",
  "ma": "Malay (Malaysian and Indonesian)",
  "nb": "Norwegian Bokmål",
  "nl": "Dutch",
  "or": "Odia",
  "ro": "Romanian",
  "sv": "Swedish",
  "sw": "Swahili",
  "tts": "Isan",
  "vi_C": "Vietnamese (Central)",
  "vi_N": "Vietnamese (Northern)",
  "vi_S": "Vietnamese (Southern)",
  "yue": "Cantonese",
  "zh": "Mandarin"
}
iso_lang_codes = list(lang_codes.keys())

# Definiendo funciones elementales para operar los datasets y su contenido
Estas vienen copiadas de la especificación de la práctica, con ligeras modificaciones.

In [2]:
def response_to_dict(ipa_list: list) -> dict:
    """Parse to dict the list of word-IPA

    Each element of text hae the format:
    [WORD][TAB][IPA]

    Parameters
    ----------
    ipa_list: list
        List with each row of ipa-dict raw dataset file

    Returns
    -------
    dict:
        A dictionary with the word as key and the phonetic
        representation as value
    """
    result = {}
    for item in ipa_list:
        item_list = item.split("\t")
        result[item_list[0]] = item_list[1]
    return result

import requests as r

def get_ipa_dict(iso_lang: str) -> dict:
    """Get ipa-dict file from Github

    Parameters:
    -----------
    iso_lang:
        Language as iso code

    Results:
    --------
    dict:
        Dictionary with words as keys and phonetic representation
        as values for a given lang code
    """
    response = r.get(f"https://raw.githubusercontent.com/open-dict-data/ipa-dict/master/data/{iso_lang}.txt")
    raw_data = response.text.split("\n")
    return response_to_dict(raw_data[:-1])

def get_ipa_transcriptions(word: str, dataset: dict) -> list[str]:
    """Search for word in a given dataset of IPA phonetics

    Given a word this function return the IPA transcriptions

    Parameters:
    -----------
    word: str
        A word to search in the dataset
    dataset: dict
        A dataset for a given language code
    Returns
    -------
    list:
        A list with all the found IPA transcriptions of the input word.
    """
    return dataset.get(word.lower(), "NOT FOUND").split(", ")

# El dataset comenzará vacío y se descargaran los corpus sobre demanda.
dataset = {}


# Definiendo el ciclo de ejecución

In [ ]:
print("Representación fonética de palabras")
print(f"Lenguas disponibles: {(iso_lang_codes)}")
print("Deja la casilla en blanco y presiona enter para salir.")

lang = "none"

while lang:
  lang = input("lang>> ")
  if lang in iso_lang_codes:

    if lang not in dataset:
      print("Corpus no encontrado. Descargando...")
      dataset.update({lang : get_ipa_dict(lang)})

    sub_dataset = dataset[lang]

    query = "none"
    while query:
      query = input(f"[{lang}]>> ")

      resultado_final = []
      for palabra in query.split():
        resultado = get_ipa_transcriptions(palabra, sub_dataset)

        if resultado[0] != "NOT FOUND":
          resultado_final += resultado
        else:
          resultado_final += "/?/"

        resultado_final += [" "]

      print("".join(resultado_final))

  elif lang:
    print("Ese lenguaje no está disponible, intenta con otro:")

print("Adios 👋🏼")
